In [1]:
import tensorflow as tf
import csv
import numpy as np 
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,Activation,Dropout,MaxPooling2D
from keras.activations import relu
from keras.optimizers import Adam
import datetime
import os

Using TensorFlow backend.


In [2]:
#we will use images 26x34x1 (1 is for grayscale images)
height = 26
width = 34
dims = 1

In [3]:
def readCsv(path):

	with open(path,'r') as f:
		#read the scv file with the dictionary format 
		reader = csv.DictReader(f)
		rows = list(reader)

	#imgs is a numpy array with all the images
	#tgs is a numpy array with the tags of the images
	imgs = np.empty((len(list(rows)),height,width, dims),dtype=np.uint8)
	tgs = np.empty((len(list(rows)),1))
		
	for row,i in zip(rows,range(len(rows))):
			
		#convert the list back to the image format
		img = row['image']
		img = img.strip('[').strip(']').split(', ')
		im = np.array(img,dtype=np.uint8)
		im = im.reshape((26,34))
		im = np.expand_dims(im, axis=2)
		imgs[i] = im

		#the tag for open is 1 and for close is 0
		tag = row['state']
		if tag == 'open':
			tgs[i] = 1
		else:
			tgs[i] = 0
	
	#shuffle the dataset
	index = np.random.permutation(imgs.shape[0])
	imgs = imgs[index]
	tgs = tgs[index]

	#return images and their respective tags
	return imgs,tgs	

In [4]:
#make the convolution neural network
def makeModel():
	model = Sequential()

	model.add(Conv2D(32, (3,3), padding = 'same', input_shape=(height,width,dims)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add(Conv2D(64, (2,2), padding= 'same'))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(128, (2,2), padding='same'))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Dropout(0.25))

	model.add(Flatten())

	model.add(Dense(512))
	model.add(Activation('relu'))

	model.add(Dense(512))
	model.add(Activation('relu'))

	model.add(Dense(1))
	model.add(Activation('sigmoid'))

	
	model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy',metrics=['accuracy'])

	return model


In [5]:
def main():

	xTrain ,yTrain = readCsv('dataset.csv')
	print (xTrain.shape[0])
	#scale the values of the images between 0 and 1
	xTrain = xTrain.astype('float32')
	xTrain /= 255

	model = makeModel()

	#do some data augmentation
	datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        )
	datagen.fit(xTrain)

	#TENSORBOARD

	log_dir = os.path.join("logs","fit",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),)
	tensorboard_callbacks=tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

	#train the model
	model.fit_generator(datagen.flow(xTrain,yTrain,batch_size=32),
						steps_per_epoch=len(xTrain) / 32, epochs=10)
	
	#save the model
	model.save('Model.hdf5')


In [6]:
if __name__ == '__main__':
	main()


2874

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
90/89 [==============================] - 5s 54ms/step - loss: 0.4347 - accuracy: 0.7857
Epoch 2/10
90/89 [==============================] - 3s 37ms/step - loss: 0.2201 - accuracy: 0.9158
Epoch 3/10
90/89 [==============================] - 3s 36ms/step - loss: 0.1792 - accuracy: 0.9384
Epoch 4/10
90/89 [==============================] - 3s 36ms/step - loss: 0.1405 - accuracy: 0.9527
Epoch 5/10
90/89 [==============================] - 3s 35ms/step - loss: 0.1453 - accuracy: 0.9523
Epoch 6/10
90/89 [==============================] - 3s 39ms/step - loss: 0.1083 - accuracy: 0.9638
Epoch 7/10
90/89 [==============================] - 3s 37ms/step - loss: 0.0904 - accuracy: 0.9701
Epoch 8/10
90/89 [==============================] - 3s 36ms/step - loss: 0.0944 - accuracy: 0.9683
Epoch 9/10
90/89 [==============================] - 3s 37ms/step - loss: 0.0635 - accuracy: 0.9760
Epoch 10